# GridWorld MDP - Proceso de Decisión de Markov

**Autor:** Ruben Casa  
**Curso:** Machine Learning  
**Fecha:** Enero 2026

Este notebook implementa un entorno GridWorld como un Proceso de Decisión de Markov (MDP),
lo resuelve utilizando Value Iteration, y analiza el comportamiento del agente desde
múltiples posiciones iniciales.

---
## Parte A – Diseño del Entorno GridWorld

Creamos un entorno GridWorld de 12×12 celdas con:
- **Obstáculos**: 6 celdas inaccesibles
- **Trampas**: 2 celdas con recompensa negativa alta (-100)
- **Meta**: 1 celda objetivo con recompensa positiva (+100)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuración del GridWorld
GRID_SIZE = 12

# Definición de tipos de celdas
EMPTY = 0
OBSTACLE = 1
TRAP = 2
GOAL = 3

# Definición de acciones
ACTIONS = {
    0: (-1, 0),  # ARRIBA
    1: (1, 0),   # ABAJO
    2: (0, -1),  # IZQUIERDA
    3: (0, 1)    # DERECHA
}
ACTION_NAMES = ['↑ Arriba', '↓ Abajo', '← Izquierda', '→ Derecha']
ACTION_SYMBOLS = ['↑', '↓', '←', '→']

# Crear el grid
grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)

# Definir obstáculos (6 celdas inaccesibles)
obstacles = [(0, 3), (1, 3), (1, 8), (3, 1), (3, 2), (5, 5)]
for obs in obstacles:
    grid[obs] = OBSTACLE

# Definir trampas (2 celdas con penalización alta)
traps = [(2, 6), (5, 10)]
for trap in traps:
    grid[trap] = TRAP

# Definir meta
goal = (11, 11)
grid[goal] = GOAL

print("=" * 60)
print("CONFIGURACIÓN DEL GRIDWORLD")
print("=" * 60)
print(f"Tamaño del grid: {GRID_SIZE} × {GRID_SIZE}")
print(f"Total de celdas: {GRID_SIZE * GRID_SIZE}")
print(f"Obstáculos ({len(obstacles)}): {obstacles}")
print(f"Trampas ({len(traps)}): {traps}")
print(f"Meta: {goal}")

In [ ]:
def visualize_gridworld(grid, title="GridWorld Environment", policy=None, values=None, 
                         trajectory=None, start_pos=None):
    """
    Visualiza el GridWorld con colores para cada tipo de celda.
    Opcionalmente muestra la política (flechas), valores, o trayectoria.
    """
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Crear mapa de colores personalizado
    colors = ['#E8F5E9', '#424242', '#F44336', '#4CAF50']  # Empty, Obstacle, Trap, Goal
    cmap = ListedColormap(colors)
    
    # Mostrar grid
    ax.imshow(grid, cmap=cmap, vmin=0, vmax=3)
    
    # Agregar valores de estado si se proporcionan
    if values is not None:
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if grid[i, j] != OBSTACLE:
                    color = 'white' if grid[i, j] in [TRAP, GOAL] else 'black'
                    ax.text(j, i, f'{values[i, j]:.1f}', ha='center', va='center',
                           fontsize=8, color=color, fontweight='bold')
    
    # Agregar flechas de política si se proporcionan
    if policy is not None:
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if grid[i, j] == EMPTY:
                    action = policy[i, j]
                    color = 'darkblue'
                    ax.text(j, i, ACTION_SYMBOLS[action], ha='center', va='center',
                           fontsize=16, color=color, fontweight='bold')
    
    # Dibujar trayectoria si se proporciona
    if trajectory is not None and len(trajectory) > 1:
        traj_y = [pos[0] for pos in trajectory]
        traj_x = [pos[1] for pos in trajectory]
        ax.plot(traj_x, traj_y, 'b-', linewidth=3, alpha=0.7, marker='o', 
                markersize=10, markerfacecolor='blue', markeredgecolor='white')
        # Marcar inicio
        ax.plot(traj_x[0], traj_y[0], 'go', markersize=20, markeredgecolor='white', 
                markeredgewidth=2, label='Inicio')
        # Marcar fin
        ax.plot(traj_x[-1], traj_y[-1], 'r*', markersize=25, markeredgecolor='white',
                markeredgewidth=2, label='Fin')
    
    # Marcar posición inicial si se proporciona
    if start_pos is not None:
        ax.plot(start_pos[1], start_pos[0], 'co', markersize=25, markeredgecolor='white',
                markeredgewidth=3, label=f'Inicio {start_pos}')
    
    # Configurar ejes
    ax.set_xticks(range(GRID_SIZE))
    ax.set_yticks(range(GRID_SIZE))
    ax.set_xticklabels(range(GRID_SIZE))
    ax.set_yticklabels(range(GRID_SIZE))
    ax.grid(True, linewidth=2, color='black')
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Columna (j)', fontsize=12)
    ax.set_ylabel('Fila (i)', fontsize=12)
    
    # Crear leyenda
    legend_elements = [
        mpatches.Patch(facecolor='#E8F5E9', edgecolor='black', label='Celda vacía'),
        mpatches.Patch(facecolor='#424242', edgecolor='black', label='Obstáculo'),
        mpatches.Patch(facecolor='#F44336', edgecolor='black', label='Trampa (-100)'),
        mpatches.Patch(facecolor='#4CAF50', edgecolor='black', label='Meta (+100)')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.02, 1))
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Visualizar el GridWorld inicial
print("\n" + "=" * 60)
print("VISUALIZACIÓN DEL GRIDWORLD")
print("=" * 60)
fig = visualize_gridworld(grid, "GridWorld 12×12 - Diseño del Entorno")

### Espacio de Estados y Acciones

**Espacio de Estados (S):**
- Total de celdas: 144 (12 × 12)
- Celdas válidas: 144 - 6 obstáculos = 138 estados
- Cada estado se representa como una tupla (fila, columna)

**Espacio de Acciones (A):**
- 4 acciones posibles: Arriba, Abajo, Izquierda, Derecha
- Las acciones son determinísticas (probabilidad de transición = 1)

**Restricciones de Movimiento:**
- El agente no puede salir del grid (bordes)
- El agente no puede atravesar obstáculos
- Si intenta un movimiento inválido, permanece en su posición actual

In [ ]:
# Definición formal del espacio de estados
valid_states = []
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if grid[i, j] != OBSTACLE:
            valid_states.append((i, j))

print("\n" + "=" * 60)
print("ESPACIO DE ESTADOS Y ACCIONES")
print("=" * 60)
print(f"\nEspacio de Estados (S):")
print(f"  - Dimensión del grid: {GRID_SIZE} × {GRID_SIZE} = {GRID_SIZE**2} celdas")
print(f"  - Obstáculos: {len(obstacles)} celdas")
print(f"  - Estados válidos: {len(valid_states)} celdas")
print(f"\nEspacio de Acciones (A):")
for i, name in enumerate(ACTION_NAMES):
    print(f"  - Acción {i}: {name} {ACTIONS[i]}")
print(f"\nRestricciones de movimiento:")
print(f"  - Bordes del grid: el agente no puede salir")
print(f"  - Obstáculos: el agente no puede atravesarlos")
print(f"  - Movimiento inválido: el agente permanece en su lugar")

---
## Parte B – Modelado como Proceso de Decisión de Markov (MDP)

Un MDP se define formalmente como una tupla (S, A, P, R, γ):

- **S (Estados)**: Conjunto de celdas válidas del GridWorld
- **A (Acciones)**: {Arriba, Abajo, Izquierda, Derecha}
- **P (Función de Transición)**: P(s'|s,a) = probabilidad de llegar a s' desde s tomando acción a
- **R (Función de Recompensa)**: R(s,a,s') = recompensa por transición
- **γ (Factor de Descuento)**: Valor entre 0 y 1 que pondera recompensas futuras

In [ ]:
# Parámetros del MDP
STEP_REWARD = -1        # Penalización por cada paso (incentiva camino corto)
TRAP_REWARD = -100      # Penalización fuerte por caer en trampa
GOAL_REWARD = 100       # Recompensa por alcanzar la meta
GAMMA = 0.95            # Factor de descuento

print("\n" + "=" * 60)
print("MODELADO COMO MDP")
print("=" * 60)
print(f"\n1. ESTADOS (S):")
print(f"   - {len(valid_states)} estados válidos")
print(f"   - Representación: (fila, columna) donde fila,columna ∈ [0, {GRID_SIZE-1}]")

print(f"\n2. ACCIONES (A):")
print(f"   - A = {{Arriba, Abajo, Izquierda, Derecha}}")
print(f"   - |A| = 4 acciones")

print(f"\n3. FUNCIÓN DE TRANSICIÓN P(s'|s,a):")
print(f"   - Transiciones determinísticas: P(s'|s,a) = 1 para el estado resultante")
print(f"   - Si el movimiento es inválido (borde/obstáculo): P(s|s,a) = 1 (permanece)")

print(f"\n4. FUNCIÓN DE RECOMPENSA R(s,a,s'):")
print(f"   - Recompensa por paso: {STEP_REWARD} (incentiva caminos cortos)")
print(f"   - Penalización trampa: {TRAP_REWARD} (estado terminal)")
print(f"   - Recompensa meta: {GOAL_REWARD} (estado terminal)")

print(f"\n5. FACTOR DE DESCUENTO (γ = {GAMMA}):")
print(f"   - Justificación: γ = {GAMMA} es un valor alto que permite al agente")
print(f"     considerar recompensas futuras importantes mientras asegura convergencia.")
print(f"   - Un valor cercano a 1 hace que el agente planifique a largo plazo,")
print(f"     lo cual es esencial para encontrar la meta evitando trampas.")
print(f"   - Si γ fuera muy bajo (ej. 0.5), el agente solo consideraría")
print(f"     recompensas inmediatas y podría tomar decisiones subóptimas.")

In [ ]:
def get_next_state(state, action):
    """
    Calcula el siguiente estado dado un estado actual y una acción.
    Implementa las restricciones de movimiento (bordes y obstáculos).
    """
    row, col = state
    d_row, d_col = ACTIONS[action]
    new_row, new_col = row + d_row, col + d_col
    
    # Verificar límites del grid
    if new_row < 0 or new_row >= GRID_SIZE or new_col < 0 or new_col >= GRID_SIZE:
        return state  # Permanece en el mismo estado
    
    # Verificar obstáculos
    if grid[new_row, new_col] == OBSTACLE:
        return state  # Permanece en el mismo estado
    
    return (new_row, new_col)

def get_reward(state, action, next_state):
    """
    Calcula la recompensa R(s,a,s') para una transición.
    """
    next_row, next_col = next_state
    
    # Recompensa por alcanzar la meta
    if grid[next_row, next_col] == GOAL:
        return GOAL_REWARD
    
    # Penalización por caer en trampa
    if grid[next_row, next_col] == TRAP:
        return TRAP_REWARD
    
    # Penalización estándar por paso
    return STEP_REWARD

def is_terminal(state):
    """
    Verifica si un estado es terminal (meta o trampa).
    """
    row, col = state
    return grid[row, col] in [GOAL, TRAP]

# Demostración de la función de transición
print("\n" + "=" * 60)
print("DEMOSTRACIÓN DE TRANSICIONES")
print("=" * 60)
test_state = (0, 0)
print(f"\nDesde el estado {test_state}:")
for action in range(4):
    next_s = get_next_state(test_state, action)
    reward = get_reward(test_state, action, next_s)
    print(f"  Acción {ACTION_NAMES[action]:15} → Estado {next_s}, Recompensa: {reward}")

---
## Parte C – Resolución del MDP con Value Iteration

Value Iteration es un algoritmo de programación dinámica que calcula la función 
de valor óptima V*(s) y extrae la política óptima π*(s).

**Ecuación de Bellman para Value Iteration:**

$$V_{k+1}(s) = \max_a \left[ R(s,a,s') + \gamma \cdot V_k(s') \right]$$

Donde:
- V_k(s) es el valor del estado s en la iteración k
- El algoritmo itera hasta que |V_{k+1} - V_k| < θ (umbral de convergencia)

In [ ]:
def value_iteration(grid, gamma=GAMMA, theta=1e-6, max_iterations=1000):
    """
    Implementa el algoritmo Value Iteration para resolver el MDP.
    
    Args:
        grid: Matriz del GridWorld
        gamma: Factor de descuento
        theta: Umbral de convergencia
        max_iterations: Máximo número de iteraciones
    
    Returns:
        V: Función de valor óptima
        policy: Política óptima
        iterations: Número de iteraciones hasta convergencia
        history: Historial del error máximo por iteración
    """
    # Inicializar función de valor a ceros
    V = np.zeros((GRID_SIZE, GRID_SIZE))
    
    # Estados terminales tienen valor fijo
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if grid[i, j] == GOAL:
                V[i, j] = GOAL_REWARD
            elif grid[i, j] == TRAP:
                V[i, j] = TRAP_REWARD
    
    history = []
    
    for iteration in range(max_iterations):
        delta = 0
        V_new = V.copy()
        
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                
                # Saltar obstáculos y estados terminales
                if grid[i, j] == OBSTACLE or is_terminal(state):
                    continue
                
                # Calcular el valor máximo sobre todas las acciones
                max_value = float('-inf')
                for action in range(4):
                    next_state = get_next_state(state, action)
                    reward = get_reward(state, action, next_state)
                    value = reward + gamma * V[next_state]
                    max_value = max(max_value, value)
                
                V_new[i, j] = max_value
                delta = max(delta, abs(V_new[i, j] - V[i, j]))
        
        V = V_new
        history.append(delta)
        
        # Verificar convergencia
        if delta < theta:
            print(f"Value Iteration convergió en {iteration + 1} iteraciones")
            print(f"Delta final: {delta:.2e}")
            break
    else:
        print(f"Value Iteration alcanzó el límite de {max_iterations} iteraciones")
    
    # Extraer política óptima
    policy = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
    
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            state = (i, j)
            
            if grid[i, j] == OBSTACLE or is_terminal(state):
                continue
            
            best_action = 0
            best_value = float('-inf')
            
            for action in range(4):
                next_state = get_next_state(state, action)
                reward = get_reward(state, action, next_state)
                value = reward + gamma * V[next_state]
                
                if value > best_value:
                    best_value = value
                    best_action = action
            
            policy[i, j] = best_action
    
    return V, policy, iteration + 1, history

In [ ]:
# Ejecutar Value Iteration
print("\n" + "=" * 60)
print("EJECUCIÓN DE VALUE ITERATION")
print("=" * 60)
print(f"\nParámetros:")
print(f"  - Factor de descuento (γ): {GAMMA}")
print(f"  - Umbral de convergencia (θ): 1e-6")
print(f"\nEjecutando...")

V_optimal, policy_optimal, num_iterations, convergence_history = value_iteration(grid)

print(f"\n{'='*60}")
print("RESULTADOS")
print("=" * 60)
print(f"\nIteraciones requeridas: {num_iterations}")
print(f"Valor máximo: {V_optimal.max():.2f}")
print(f"Valor mínimo (excluyendo obstáculos): {V_optimal[grid != OBSTACLE].min():.2f}")

In [ ]:
# Visualizar curva de convergencia
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(convergence_history, 'b-', linewidth=2)
ax.axhline(y=1e-6, color='r', linestyle='--', label='Umbral θ = 1e-6')
ax.set_xlabel('Iteración', fontsize=12)
ax.set_ylabel('Delta (Cambio máximo)', fontsize=12)
ax.set_title('Convergencia de Value Iteration', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar función de valor
print("\n" + "=" * 60)
print("FUNCIÓN DE VALOR ÓPTIMA V*(s)")
print("=" * 60)

fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Crear máscara para obstáculos
V_display = V_optimal.copy()
V_display[grid == OBSTACLE] = np.nan

# Mostrar heatmap
im = ax.imshow(V_display, cmap='RdYlGn', vmin=-100, vmax=100)

# Agregar valores en cada celda
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if grid[i, j] != OBSTACLE:
            color = 'white' if abs(V_optimal[i, j]) > 50 else 'black'
            ax.text(j, i, f'{V_optimal[i, j]:.1f}', ha='center', va='center',
                   fontsize=9, color=color, fontweight='bold')
        else:
            ax.text(j, i, 'X', ha='center', va='center',
                   fontsize=14, color='white', fontweight='bold')

ax.set_xticks(range(GRID_SIZE))
ax.set_yticks(range(GRID_SIZE))
ax.set_title('Función de Valor Óptima V*(s)', fontsize=14, fontweight='bold')
ax.set_xlabel('Columna', fontsize=12)
ax.set_ylabel('Fila', fontsize=12)
plt.colorbar(im, ax=ax, label='Valor del Estado')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar política óptima
print("\n" + "=" * 60)
print("POLÍTICA ÓPTIMA π*(s)")
print("=" * 60)

fig = visualize_gridworld(grid, "Política Óptima π*(s)", policy=policy_optimal)

In [ ]:
# Tabla de política óptima
print("\nPolítica óptima (direcciones):")
print("-" * 50)
for i in range(GRID_SIZE):
    row_str = ""
    for j in range(GRID_SIZE):
        if grid[i, j] == OBSTACLE:
            row_str += " X "
        elif grid[i, j] == GOAL:
            row_str += " G "
        elif grid[i, j] == TRAP:
            row_str += " T "
        else:
            row_str += f" {ACTION_SYMBOLS[policy_optimal[i, j]]} "
    print(row_str)

---
## Parte D – Análisis desde Múltiples Estados Iniciales

Ejecutamos el agente desde dos posiciones iniciales distintas para analizar:
- Trayectorias seguidas
- Número de pasos hasta la meta
- Influencia de obstáculos y trampas en la política

In [ ]:
def execute_policy(start_state, policy, max_steps=100):
    """
    Ejecuta la política desde un estado inicial hasta llegar a un estado terminal
    o alcanzar el máximo de pasos.
    
    Returns:
        trajectory: Lista de estados visitados
        total_reward: Recompensa acumulada
        terminated: Si terminó en meta o trampa
    """
    trajectory = [start_state]
    current_state = start_state
    total_reward = 0
    
    for step in range(max_steps):
        if is_terminal(current_state):
            break
        
        action = policy[current_state]
        next_state = get_next_state(current_state, action)
        reward = get_reward(current_state, action, next_state)
        
        total_reward += reward
        trajectory.append(next_state)
        current_state = next_state
    
    # Determinar estado final
    final_row, final_col = current_state
    if grid[final_row, final_col] == GOAL:
        terminated = "META"
    elif grid[final_row, final_col] == TRAP:
        terminated = "TRAMPA"
    else:
        terminated = "MAX_PASOS"
    
    return trajectory, total_reward, terminated

In [ ]:
# Definir posiciones iniciales
start_positions = [
    (0, 0),   # Esquina superior izquierda
    (11, 0),  # Esquina inferior izquierda
]

print("\n" + "=" * 60)
print("ANÁLISIS DESDE MÚLTIPLES ESTADOS INICIALES")
print("=" * 60)

results = []

for i, start in enumerate(start_positions):
    print(f"\n{'─' * 40}")
    print(f"EJECUCIÓN {i + 1}: Inicio en {start}")
    print(f"{'─' * 40}")
    
    trajectory, total_reward, terminated = execute_policy(start, policy_optimal)
    
    print(f"Estado final: {terminated}")
    print(f"Número de pasos: {len(trajectory) - 1}")
    print(f"Recompensa total: {total_reward:.2f}")
    print(f"\nTrayectoria:")
    
    # Mostrar trayectoria de forma legible
    for step, state in enumerate(trajectory):
        if step == 0:
            print(f"  Paso {step}: {state} (INICIO)")
        elif step == len(trajectory) - 1:
            print(f"  Paso {step}: {state} ({terminated})")
        else:
            action = policy_optimal[trajectory[step-1]]
            print(f"  Paso {step}: {state} (acción: {ACTION_SYMBOLS[action]})")
    
    results.append({
        'start': start,
        'trajectory': trajectory,
        'steps': len(trajectory) - 1,
        'reward': total_reward,
        'terminated': terminated
    })

In [ ]:
# Visualizar trayectorias
fig, axes = plt.subplots(1, 2, figsize=(20, 9))

for idx, result in enumerate(results):
    ax = axes[idx]
    
    # Crear mapa de colores
    colors = ['#E8F5E9', '#424242', '#F44336', '#4CAF50']
    cmap = ListedColormap(colors)
    
    ax.imshow(grid, cmap=cmap, vmin=0, vmax=3)
    
    # Dibujar trayectoria
    trajectory = result['trajectory']
    if len(trajectory) > 1:
        traj_y = [pos[0] for pos in trajectory]
        traj_x = [pos[1] for pos in trajectory]
        
        # Línea de trayectoria
        ax.plot(traj_x, traj_y, 'b-', linewidth=4, alpha=0.7)
        
        # Puntos de la trayectoria
        for i, (x, y) in enumerate(zip(traj_x, traj_y)):
            if i == 0:
                ax.plot(x, y, 'go', markersize=25, markeredgecolor='white', 
                       markeredgewidth=3, zorder=5)
                ax.text(x, y, 'S', ha='center', va='center', fontsize=12, 
                       color='white', fontweight='bold', zorder=6)
            elif i == len(trajectory) - 1:
                if result['terminated'] == 'META':
                    ax.plot(x, y, 'g*', markersize=30, markeredgecolor='white',
                           markeredgewidth=2, zorder=5)
                else:
                    ax.plot(x, y, 'r*', markersize=30, markeredgecolor='white',
                           markeredgewidth=2, zorder=5)
            else:
                ax.plot(x, y, 'bo', markersize=12, markeredgecolor='white', 
                       markeredgewidth=1, alpha=0.8, zorder=4)
    
    # Configurar ejes
    ax.set_xticks(range(GRID_SIZE))
    ax.set_yticks(range(GRID_SIZE))
    ax.grid(True, linewidth=1, color='black', alpha=0.5)
    ax.set_title(f"Trayectoria desde {result['start']}\n"
                 f"Pasos: {result['steps']} | Recompensa: {result['reward']:.0f} | {result['terminated']}", 
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Columna', fontsize=12)
    ax.set_ylabel('Fila', fontsize=12)

# Agregar leyenda
legend_elements = [
    mpatches.Patch(facecolor='#E8F5E9', edgecolor='black', label='Celda vacía'),
    mpatches.Patch(facecolor='#424242', edgecolor='black', label='Obstáculo'),
    mpatches.Patch(facecolor='#F44336', edgecolor='black', label='Trampa'),
    mpatches.Patch(facecolor='#4CAF50', edgecolor='black', label='Meta'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='green', 
               markersize=15, label='Inicio'),
    plt.Line2D([0], [0], marker='o', color='b', markerfacecolor='blue', 
               markersize=10, label='Trayectoria'),
]
fig.legend(handles=legend_elements, loc='lower center', ncol=6, bbox_to_anchor=(0.5, -0.02))

plt.tight_layout()
plt.show()

In [ ]:
# Tabla comparativa
print("\n" + "=" * 60)
print("COMPARACIÓN DE TRAYECTORIAS")
print("=" * 60)
print(f"\n{'Posición Inicial':<20} {'Pasos':<10} {'Recompensa':<15} {'Resultado':<15}")
print("-" * 60)
for result in results:
    print(f"{str(result['start']):<20} {result['steps']:<10} {result['reward']:<15.2f} {result['terminated']:<15}")

In [ ]:
# Análisis detallado
print("\n" + "=" * 60)
print("ANÁLISIS DETALLADO")
print("=" * 60)

print(f"""
INFLUENCIA DE OBSTÁCULOS Y TRAMPAS EN LA POLÍTICA:

1. OBSTÁCULOS:
   - Los obstáculos en las posiciones {obstacles} actúan como barreras físicas
   - Fuerzan al agente a rodearlos, aumentando la longitud del camino
   - La política óptima encuentra rutas que minimizan el impacto de los obstáculos

2. TRAMPAS:
   - Las trampas en {traps} tienen una penalización de {TRAP_REWARD}
   - La política óptima dirige al agente LEJOS de las trampas
   - El valor de los estados cercanos a las trampas es más bajo
   - El agente prefiere caminos más largos si evitan las trampas

3. COMPARACIÓN DE TRAYECTORIAS:
   - Desde (0,0): El agente debe navegar evitando los obstáculos en columnas 1-3
     y la trampa en (2,6). Toma {results[0]['steps']} pasos.
   - Desde (11,0): El agente tiene un camino más directo hacia la meta en (11,11).
     Toma {results[1]['steps']} pasos.

4. DIFERENCIA EN PASOS:
   - La diferencia de {abs(results[0]['steps'] - results[1]['steps'])} pasos se debe a:
     * Distancia inicial a la meta
     * Obstáculos y trampas en el camino
     * Configuración específica del entorno
""")

---
## Conclusión Reflexiva

### ¿Cómo influye el diseño del entorno en la política óptima?

El diseño del entorno tiene un impacto fundamental en la política óptima. La ubicación 
de obstáculos obliga al agente a tomar rutas alternativas, mientras que las trampas 
crean "zonas de peligro" que la política aprende a evitar. En nuestro GridWorld, la 
trampa en (2,6) hace que los estados cercanos tengan valores negativos, dirigiendo 
al agente hacia caminos más seguros aunque sean más largos.

### ¿Por qué un mismo MDP puede generar trayectorias distintas desde diferentes estados iniciales?

Aunque el MDP y la política óptima son únicos, la trayectoria que sigue el agente 
depende completamente de su estado inicial. Cada estado tiene una acción óptima 
específica que maximiza la recompensa esperada desde ESE punto en adelante. Por 
tanto, diferentes puntos de partida implican diferentes secuencias de estados 
visitados, aunque todas converjan hacia la meta siguiendo la misma política.

### ¿Qué ventajas ofrece el modelado con MDP frente a reglas heurísticas?

El modelado con MDP ofrece varias ventajas clave: (1) **Optimalidad garantizada** - 
Value Iteration encuentra la política que maximiza la recompensa esperada, algo que 
las heurísticas no pueden garantizar. (2) **Adaptabilidad** - cambiar las recompensas 
o el entorno automáticamente produce una nueva política óptima sin reprogramación 
manual. (3) **Manejo de incertidumbre** - los MDP pueden modelar transiciones 
estocásticas, mientras que las heurísticas suelen asumir determinismo. (4) 
**Planificación a largo plazo** - el factor de descuento permite balancear recompensas 
inmediatas y futuras de forma sistemática.

In [ ]:
print("\n" + "=" * 60)
print("CONCLUSIÓN REFLEXIVA")
print("=" * 60)
print("""
1. INFLUENCIA DEL DISEÑO DEL ENTORNO:
   El diseño del entorno es determinante para la política óptima. Los obstáculos
   crean barreras que fuerzan rutas alternativas, mientras que las trampas generan
   "campos de repulsión" en la función de valor. La política óptima emerge como
   un balance entre alcanzar la meta rápidamente y evitar penalizaciones.

2. TRAYECTORIAS DISTINTAS DESDE DIFERENTES ESTADOS:
   Aunque la política π*(s) es única para todo el MDP, la trayectoria depende del
   estado inicial. Cada estado tiene su acción óptima local que forma parte de la
   solución global. Diferentes puntos de partida generan diferentes caminos hacia
   la meta, todos siguiendo la misma política pero visitando distintos estados.

3. VENTAJAS DEL MODELADO MDP vs REGLAS HEURÍSTICAS:
   - Optimalidad matemática garantizada (no aproximaciones ad-hoc)
   - Adaptabilidad automática a cambios en el entorno o recompensas
   - Capacidad de modelar incertidumbre mediante transiciones estocásticas
   - Planificación a largo plazo sistemática mediante el factor de descuento
   - Fundamentación teórica sólida que permite análisis formal
""")

print("\n" + "=" * 60)
print("FIN DEL ANÁLISIS")
print("=" * 60)